# BET Final Code
## Jamie DeCoster 112020536

# Specific surface area

In [1]:
def N2ads(myfilename):
    """This function opens a CSV file of N2 adsorption data and then plots and saves the P/P0 and adsorbed volume data as numpy 
    arrays. The file inputted must have the data listed in columns with the first column being P/P0 and the second being adsorbed 
    volume. The file can contain multiple experiments of P/P0 and adsorbed volumes but they must alternate in that order.
    inputs:
    myfilename: .csv file of P/P0 and adsorbed volume in cm^3/g
    
    outputs:
    P_P0: numpy array of the relative pressure data
    V_adsorbed: numpy array of adsorbed gas"""
    
    #Use pandas to save the adsorption data into a data frame
    df_N2ads=pd.read_csv(myfilename)

    #Save the column titles of the data
    column_titles=df_N2ads.columns

    #Save a variable to represent the number of samples (each sample has 2 columns of data)
    n=int(len(column_titles)/2)

    #Preallocate the arrays
    P_P0=np.zeros((df_N2ads.shape[0],n))
    V_adsorbed=np.zeros((df_N2ads.shape[0],n))

    #Save a color map for plotting
    mycolors=plt.cm.rainbow(np.linspace(0,1,n))

    #Create a figure to plot the data
    plt.figure(figsize=(8,5),dpi=300,facecolor='white')

    #Create a vector of markers for plotting
    mymarkers = ['o', 's', '*', 'v', '^', 'D', 'h', 'x', '+', '8', 'p', '<', '>', 'd', 'H']

    
    #The count variable is used to iterate through the samples (1 to n) and thus store data for each sample in the corresponding
    #column of the numpy arrays (each column in P/P0 and V_adsorbed corresponds to a different sample).
    count=0
    
    #The i in the for loop allows us to iterate through the column title list. In the data being imported every 2 columns 
    #corresponds to 1 sample (allows for proper storage of the data as either P/P0 or V_adsorbed)
    for i in range(0,len(column_titles),2): 
        #Save the data in a numpy array by indexing into the data frame using the column titles to get 
        #the P/P0 and V_adsorbed for that sample
        P_P0[:,count]=np.array(df_N2ads[column_titles[i]])
        V_adsorbed[:,count]=np.array(df_N2ads[column_titles[i+1]])

        #Plot the data
        plt.plot(P_P0[:,count],V_adsorbed[:,count],c=mycolors[count],marker=mymarkers[count],label=column_titles[i+1])

        #Iterate to the next sample
        count+=1

    #Add proper information to the plot
    plt.xlabel("P/$P_0$")
    plt.ylabel("Volume Adsorbed ($cm^{3}$/g)")
    plt.title("$N_2$-Adsorption Isotherms")
    plt.legend()
    plt.savefig("Adsorption Isotherms.png",dpi=300)
    plt.close()
    
    return P_P0, V_adsorbed

In [2]:
def SSA_cat(P,V,sigma0=16.2e-20):
    """Calculates the specific surface area of the catalyst
    inputs:
    P: relative pressure (P/P0)
    V: volume of gas adsorbed per gram of catalyst in cm^3/g
    sigma0: cross sectional area of adsorbed gas in m^2 (16.2e-20 for nitrogen)
    
    output:
    SSA: Specific surface area"""

    #Preallocate Avogadro's number and the volume of gas per mol in m^3/mol as these are constants
    N_avo=6.022e23 #1/mol
    mol_vol=0.0224 #m^3/mol
    
    #Preallocate the lists to be empty
    x=[]
    y=[]

    #Slice the x and y data so it is only data corresponding to monolayer formation (0.025 < P/P0 < 0.3)
    for i in range(len(P)):
        if P[i]<0.025:
            continue
        if P[i]>0.3:
            break
        x.append(P[i])
        y.append(P[i]/(V[i]*(1-P[i])))

    #Save as a numpy array
    x=np.array(x)
    y=np.array(y)
    
    #Find the linear fit of the data
    SSA_linreg=stats.linregress(x,y)
    m=SSA_linreg.slope
    b=SSA_linreg.intercept

    #Calculate the points for the linear fit
    x1=np.linspace(x[0],x[len(x)-1],100)
    y1=m*x1+b
    
    #Determine the R^2 value 
    res=y-(m*x+b)
    SSE=sum((res)**2)
    SST=sum((y-np.mean(y))**2)
    R_square=1.0-(SSE/SST)
    
    #Determine the RMS
    rms=np.sqrt(sum((pow((np.array(m*x+b)-y),2)))/len(y))
    
    #Plot the experimental data and the line of fit and save as a png file
    #Create a figure
    plt.figure(figsize=(8,5),dpi=300,facecolor='white')
    
    #Plot data
    plt.plot(x,y,'ro',label='Experimental')
    plt.plot(x1,y1,'b-',label="Linear Regression")
    
    #Label the figure
    plt.xlabel("P/$P_0$")
    plt.ylabel("(P/$P_0$)/(v*(1-P/$P_0$))")
    plt.title("$N_2$-Adsorption Isotherms: Monolayer Capacity for Sample {}".format(runningcount))
    plt.legend()
    
    #Add the equation and R^2 value to the plot
    plt.text(0.2,min(y),'y = {:.4f}x + {:.4f}\n$R^2$ = {:.4f}\nRMS = {:.4e}'.format(m,b,R_square,rms),fontsize=14, bbox=dict(facecolor='grey',alpha=0.2))
    
    #Save the figure
    plt.savefig("Linear Fit of Adsorption data {}.png".format(runningcount),dpi=300)
    plt.close()

    #Find the monolayer capacity
    vm=1/(b+m)

    #Then calculate the surface area in m^2/g
    SSA=vm*(1/100)**3*sigma0*N_avo/mol_vol
    
    return SSA

In [3]:
#Import the necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

#Preallocate running count as 1, this allows you to save multiple samples' plots as png images
runningcount=1

#Now we have two functions, one that opens and saves data from a csv file and plots the results. And a second function
#which finds the BET SSA of the catalysts. Now we need a script which can call these functions

#First open the file and store the data using the first function
P_P0,V_adsorbed=N2ads("N2 Adsorption-Jamie DeCoster.csv")

#Save the number of columns of the data returned from the N2ads function (this is the number of samples)
q=P_P0.shape[1]

#Preallocate the solution array
SSA_CeO2=np.zeros(q)

#Save the data to a file
myfile2=open("SSA.txt",'w')
myfile2.write("Sample Number      SSA (m^2/g)\n")

#Loop through the number of samples and calculate the specific surface area for each by calling the second function
for i in range(q):
    SSA_CeO2[i]=SSA_cat(P_P0[:,i],V_adsorbed[:,i])
    myfile2.write("{:7.0f} {:18.2f}\n".format(i+1,SSA_CeO2[i]))
    
    #Increase the running count with each time SSA_cat is called to save the plot for each sample
    runningcount+=1

#Close the file    
myfile2.close()